##### Estratégias no artigo do Davi Pereira dos Santos

- euc = euclidiana
- man = Manhattan
- ent = entropia
- acu = acuracia


- ATUeuc
- ATUman 


- EERacu 
- EERent 


- HS 


- HTUeuc
- HTUman 


- Mar
- Rnd 
- SGmulti 


- TUeuc
- TUman 


- DWeuc 
- DWman
_____

## Descartada
- SVMsim
- SVMbal
- QBC

ID_ATU(x) = (1/|U| SUM(sim(x,u)))^alpha (1/|U| SUM(sim(x,u)))^-delta

### setup KNN

In [7]:
import pandas as pd
import numpy as np

In [9]:
# https://archive.ics.uci.edu/ml/datasets/Balance+Scale

df = pd.read_csv('balance-scale.data', header = None, names = ['CN','LW','LD','RW','RD'])
df = df[['LW','LD','RW','RD','CN']]
df.head()

,LW,LD,RW,RD,CN
0,1,1,1,1,B
1,1,1,1,2,R
2,1,1,1,3,R
3,1,1,1,4,R
4,1,1,1,5,R


In [18]:
# normalização dos dados

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop('CN', axis = 1))
df_scaled = scaler.transform(df.drop('CN',axis=1))
df_scaled = pd.DataFrame(df_scaled, columns= df.columns[1:])
df_scaled.head()

,LD,RW,RD,CN
0,-1.414214,-1.414214,-1.414214,-1.414214
1,-1.414214,-1.414214,-1.414214,-0.707107
2,-1.414214,-1.414214,-1.414214,0.000000
3,-1.414214,-1.414214,-1.414214,0.707107
4,-1.414214,-1.414214,-1.414214,1.414214


In [19]:
from sklearn.model_selection import train_test_split

X = df_feat
y = df['CN']

X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X, y, test_size=0.3, random_state = 1)

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_labeled, y_labeled)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

## Algoritmo 2: Amostragem ativa baseada na reserva de exemplos

In [238]:
def get_pool():
    from sklearn.linear_model import LinearRegression

    data = pd.read_csv('preco-imovel.csv', sep=';')

    data['log_Valor'] = np.log(data['Valor'])
    data['log_Area'] = np.log(data['Area'])
    data['log_Dist_Praia'] = np.log(data['Dist_Praia'] + 1)
    data['log_Dist_Farmacia'] = np.log(data['Dist_Farmacia'] + 1)


    X = data[['log_Area', 'log_Dist_Praia', 'log_Dist_Farmacia']]
    y = data['log_Valor']

    X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X, y, test_size=0.2, random_state=1)
    
    l = {'X':X_labeled, 'y':y_labeled}
    u = {'X':X_unlabeled}
    
    return l, u

In [61]:
def induct_model(l):
    model = LinearRegression()
    model.fit(l["X"], l["y"])
    return model

In [236]:
# predict X_unlabeled
# take argmax of predict values
# i = index of max value
# extract index

def arg_max(model, u):
    pred = model.predict(u["X"])
    max_v = np.amax(pred)
    i = pred == max_v
    i = np.nonzero(i)
    i[0][0]
    ex = {'index':i[0][0], 'oracle':pred[i[0][0]]}
    return ex

In [ ]:
def concat_l(l, ex):
    l2 = l
    locx = u["X"].iloc[ex["index"]]
    locy = ex["oracle"]

    l2["X"].append(locx, ignore_index=True)
    l2["y"].append(pd.Series([locy]), ignore_index=True)
    
    return l2

def remove_u(u, ex):
    

In [105]:
def active_sample(u, l, phi, q, c):
    if c == 0:
        return l
    else:
        model = phi(l);
        ex = q(model,u);
        l2 = concat_l(l,ex)
        u2 = remove_u(u,ex)
        return active_sample(u2, l2, phi, q, c-1)

In [294]:
u["X"].iloc[ex["index"]]

log_Area             7.600902
log_Dist_Praia       0.160679
log_Dist_Farmacia    0.268945
Name: 3693, dtype: float64

In [244]:
l, u = get_pool()
phi = induct_model
q = arg_max
c = 5

final_l = active_sample(u, l, phi, q, c)
final_l

{'X':       log_Area  log_Dist_Praia  log_Dist_Farmacia
 2764  4.882802        0.738724           0.622371
 4767  5.214936        0.873405           0.202240
 3814  4.158883        1.952026           0.188169
 3499  4.025352        2.852740           0.475234
 2735  4.499810        0.290537           0.188499
 ...        ...             ...                ...
 448   4.007333        2.032718           0.281887
 921   4.077537        1.965363           0.597445
 4087  4.700480        2.065791           0.404373
 1242  4.382027        0.808076           0.133670
 2242  3.688879        0.120906           0.510717
 
 [1000 rows x 3 columns]}